In [ ]:
from google.colab import files
uploaded=files.upload()


Saving ID3 - Sheet1.csv to ID3 - Sheet1.csv


In [ ]:
import pandas as pd
df=pd.read_csv('ID3 - Sheet1.csv')
print(df)

     Outlook  Temp Humidity    Wind PlayTennis
0      Sunny   Hot     High    Weak         No
1      Sunny   Hot     High  Strong         No
2   Overcast   Hot     High    Weak        Yes
3       Rain  Mild     High    Weak        Yes
4       Rain  Cool   Normal    Weak        Yes
5       Rain  Cool   Normal  Strong         No
6   Overcast  Cool   Normal  Strong        Yes
7      Sunny  Mild     High    Weak         No
8      Sunny  Cool   Normal    Weak        Yes
9       Rain  Mild   Normal    Weak        Yes
10     Sunny  Mild   Normal  Strong        Yes
11  Overcast  Mild     High  Strong        Yes
12  Overcast   Hot   Normal    Weak        Yes
13      Rain  Mild     High  Strong         No


In [ ]:
def entropy(probs):
  import math
  return sum(-prob*math.log(prob,2) for prob in probs)


In [ ]:
def entropy_of_list(a_list):
  from collections import Counter
  cnt=Counter(x for x in a_list)
  print(cnt)
  num_instances=len(a_list)
  probs=[x/num_instances for x in cnt.values()]
  print(num_instances)
  print(probs)
  return entropy(probs)
total_entropy=entropy_of_list(df['PlayTennis'])
print(total_entropy)


Counter({'Yes': 9, 'No': 5})
14
[0.35714285714285715, 0.6428571428571429]
0.9402859586706309


In [ ]:
def information_gain(df,split_attribute_name,target_attribute_name,trace=0):
  df_split=df.groupby(split_attribute_name)
  print(df_split)
  for name,group in df_split:
    print("Group",group)
    nobs=len(df.index)*1.0
    print(nobs)
    print("NOBS",nobs)
    df_agg_ent=df_split.agg({target_attribute_name: [entropy_of_list,lambda x: len(x)/nobs]})[target_attribute_name]
    avg_info=sum(df_agg_ent['entropy_of_list'] * df_agg_ent['<lambda_0>'])
    old_entropy=entropy_of_list(df[target_attribute_name])
    return old_entropy-avg_info


In [ ]:
def id3DT(df,target_attribute_name,attribute_name,default_class=None):
  from collections import Counter
  cnt=Counter(x for x in df[target_attribute_name])
  if len(cnt)==1:
    return next(iter(cnt))
  elif df.empty or (not attribute_name):
    return default_class
  else:
    default_class=max(cnt.keys())
    print("attribute_names:",attribute_names)
    gainz=[information_gain(df,attr,target_attribute_name) for attr in attribute_names]
    index_of_max=gainz.index(max(gainz))
    best_attr=attribute_names[index_of_max]
    tree={best_attr:{}}
    remaining_attributes_names=[i for i in attribute_names if i !=best_attr]
    for attr_val,data_subset in df.groupby(best_attr):
      subtree=id3DT(data_subset,target_attribute_name,remaining_attributes_names,default_class)
      tree[best_attr][attr_val]=subtree
  return tree
attribute_names=list(df.columns)
attribute_names.remove('PlayTennis')

In [ ]:
# @title
from pprint import pprint
tree=id3DT(df,'PlayTennis',attribute_names)
print("The Resultant Decision Tree is ")
pprint(tree)
attribute=next(iter(tree))
print("Best Attribute: \n",attribute)
print("Tree Keys'n",tree[attribute].keys())


attribute_names: ['Outlook', 'Temp', 'Humidity', 'Wind']
Group      Outlook  Temp Humidity    Wind PlayTennis
2   Overcast   Hot     High    Weak        Yes
6   Overcast  Cool   Normal  Strong        Yes
11  Overcast  Mild     High  Strong        Yes
12  Overcast   Hot   Normal    Weak        Yes
14.0
NOBS 14.0
Counter({'Yes': 4})
4
[1.0]
Counter({'Yes': 3, 'No': 2})
5
[0.6, 0.4]
Counter({'No': 3, 'Yes': 2})
5
[0.6, 0.4]
Counter({'Yes': 9, 'No': 5})
14
[0.35714285714285715, 0.6428571428571429]
Group     Outlook  Temp Humidity    Wind PlayTennis
4      Rain  Cool   Normal    Weak        Yes
5      Rain  Cool   Normal  Strong         No
6  Overcast  Cool   Normal  Strong        Yes
8     Sunny  Cool   Normal    Weak        Yes
14.0
NOBS 14.0
Counter({'Yes': 3, 'No': 1})
4
[0.75, 0.25]
Counter({'No': 2, 'Yes': 2})
4
[0.5, 0.5]
Counter({'Yes': 4, 'No': 2})
6
[0.6666666666666666, 0.3333333333333333]
Counter({'Yes': 9, 'No': 5})
14
[0.35714285714285715, 0.6428571428571429]
Group      Outlook

In [ ]:
def classify(instance,tree,default=None):
  attribute=next(iter(tree))
  print("Key:",tree.keys())
  print("Attribute",attribute)
  if instance[attribute] in tree[attribute].keys():
    result=tree[attribute][instance[attribute]]
    print("Instance Attribute:",instance[attribute], "Tree Keys:",tree[attribute].keys())
    if isinstance (result,dict):
      return classify(instance,result)
    else:
      return result
  else:
    return default
tree1={'Outlook':['Rain','Sunny'],'Temp':['Mild','Hot'],'Humidity':['High','High'],'Wind':['Weak','Strong']}
df2=pd.DataFrame(tree1)
df2['Predicted']=df2.apply(classify,axis=1,args=(tree,'No'))
print(df2)


Key: dict_keys(['Outlook'])
Attribute Outlook
Instance Attribute: Rain Tree Keys: dict_keys(['Overcast', 'Rain', 'Sunny'])
Key: dict_keys(['Wind'])
Attribute Wind
Instance Attribute: Weak Tree Keys: dict_keys(['Strong', 'Weak'])
Key: dict_keys(['Outlook'])
Attribute Outlook
Instance Attribute: Sunny Tree Keys: dict_keys(['Overcast', 'Rain', 'Sunny'])
Key: dict_keys(['Humidity'])
Attribute Humidity
Instance Attribute: High Tree Keys: dict_keys(['High', 'Normal'])
  Outlook  Temp Humidity    Wind Predicted
0    Rain  Mild     High    Weak       Yes
1   Sunny   Hot     High  Strong        No
